# CARGA DEL DATASET Y LIBRERIAS

## Importacion de Librerias

In [231]:
import pandas as pd
import numpy as np
import re

#Seteos del notebook
#pd.set_option('display.max_colwidth', 40)
#pd.options.display.min_rows
#pd.set_option('display.width', 1000)

## Carga del dataset crudo para su tratamiento

In [232]:
df_steam_games = pd.read_json('../raw_data/output_steam_games.json', lines=True)

# LIMPIEZA Y TRANSFORMACION DEL DATASET CRUDO

## Exploracion Basica

In [233]:
print('columnas en df_steam_games: ','[' + ', '.join(df_steam_games.keys()) + ']')

columnas en df_steam_games:  [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]


In [234]:
print('Numero de filas y campos: ',df_steam_games.shape)
print('Tipos de las columnas:')
print(df_steam_games.dtypes)
print('\n')

print('Datos faltantes:')
print(df_steam_games.isnull().sum())
df_steam_games.head(3)

#df_user_reviews_userdata_prepared[['user_id','user_url']].head(5)

Numero de filas y campos:  (120445, 13)
Tipos de las columnas:


publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price            object
early_access    float64
id              float64
developer        object
dtype: object


Datos faltantes:
publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Al parecer hay muchos valores faltantes en el dataset

## Eliminacion de Datos Faltantes

Eliminamos filas completamente vacias y reseteamos el indice

In [235]:
print('Numero de filas y campos antes de la eliminacion: ',df_steam_games.shape)
df_steam_games = df_steam_games.dropna(how='all') #elimina filas con valores faltantes en todas las columnas
print('Numero de filas y campos despues de la eliminacion: ',df_steam_games.shape)
df_steam_games.reset_index(inplace = True, drop = True)

print('Datos faltantes:')
print(df_steam_games.isnull().sum())

Numero de filas y campos antes de la eliminacion:  (120445, 13)
Numero de filas y campos despues de la eliminacion:  (32135, 13)
Datos faltantes:
publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


Eliminamos todas las filas con valores faltantes en las columnas 'id' y 'price' Y reseteamos el indice

In [236]:
df_steam_games = df_steam_games.dropna(subset=['price'])
df_steam_games = df_steam_games.dropna(subset=['id'])
df_steam_games.reset_index(inplace = True, drop = True)

print(df_steam_games.isnull().sum())

publisher       7773
genres          3135
app_name           0
title           1932
url                0
release_date    1936
tags             130
reviews_url        0
specs            655
price              0
early_access       0
id                 0
developer       3156
dtype: int64


## Tratamiento del campo 'id'

### Exploramos 'id'

In [237]:
print('columnas en df_steam_games: ','[' + ', '.join(df_steam_games.keys()) + ']')
print("Tipo de dato en 'id':",df_steam_games['id'].dtype)


columnas en df_steam_games:  [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Tipo de dato en 'id': float64


Verificamos si hay valores unicos no numéricos en  'id'

In [238]:
valores_no_numericos = [valor for valor in df_steam_games['id'].unique() if not str(valor).replace('.', '', 1).isdigit()] 
print("Valores no numericos en 'id': ", valores_no_numericos)

Valores no numericos en 'id':  []


Todos los valores en la columna 'id' deben tener el formato "xx.00". Verificamos si hay valores que no cumplan tal formato

In [239]:
# Busca valores en 'id' que no sigan el formato deseado
valores_no_validos = df_steam_games[~df_steam_games['id'].astype(str).str.contains(r'^\d+\.0$')]
#valores_no_validos = df_steam_games_userdata_prepared[~df_steam_games_userdata_prepared['id'].astype(str).str.match(r'^\d+\.\d{1}$')]

print('Ides sin el formato "xx.00": ', len(valores_no_validos['id'])) #muestra la cantidad de valores sin el formato

Ides sin el formato "xx.00":  0


Los ides deben ser unicos, por lo que verificamos si existen valores repetidos

In [240]:
#Muestra los valores unicos en 'id'
ides = df_steam_games['id'] #muestra todos los valores únicos en la columna 'price'
print("Total de valores en 'id': ", len(ides))

ides_unicos = df_steam_games['id'].unique() #muestra valores únicos en la columna 'price'
print("Número de valores únicos en 'id': ", len(ides_unicos))


Total de valores en 'id':  30756
Número de valores únicos en 'id':  30755


Inferimos que existe un valor numerico repetido en 'id'. Lo capturamos y mostramos las filas implicadas 

In [241]:
# Captura el valor repetido en 'id'
id_repetido = df_steam_games['id'].value_counts().index[df_steam_games['id'].value_counts() > 1]

#Captura los indices de las filas que tienen el indice repetido
indices_filas_repetidas = df_steam_games[df_steam_games['id'] == id_repetido[0]].index.tolist()

# Captura todas las filas con el valor repetido en 'id'
filas_con_ides_rep = df_steam_games.iloc[indices_filas_repetidas]

#Imprime las capturas
print(f"El id {id_repetido[0]} se repite en las filas con indices: {indices_filas_repetidas}.")
filas_con_ides_rep

El id 612880.0 se repite en las filas con indices: [13286, 13926].


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13286,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
13926,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


### Eliminamos duplicado y reseteamos el indice

Eliminamos la fila de indice 13926, reseteamos el indice y verificamos

In [242]:
print('Numero de filas y campos antes de la eliminacion: ',df_steam_games.shape)
df_steam_games = df_steam_games.drop(13926)
print('Numero de filas y campos despues de la eliminacion: ',df_steam_games.shape)# ELimina la fila

ides = df_steam_games['id'] #muestra todos lo valores en la columna 'id'
print("Total de valores en 'id': ", len(ides))

ides_unicos = df_steam_games['id'].unique() #muestra valores únicos en la columna 'id'
print("Número de valores únicos en 'id': ", len(ides_unicos))

# Muestra las filas en el rango entre 13023 y 13027
rango_indices = range(13924, 13928) 
filas_en_rango = df_steam_games.iloc[rango_indices][['id','price']] #filtra las filas 
filas_en_rango

Numero de filas y campos antes de la eliminacion:  (30756, 13)
Numero de filas y campos despues de la eliminacion:  (30755, 13)
Total de valores en 'id':  30755
Número de valores únicos en 'id':  30755


,id,price
13924,743520.0,2.99
13925,566680.0,1.99
13927,497800.0,9.99
13928,570540.0,9.99


Reseteamos el indice

In [243]:
df_steam_games.reset_index(inplace = True, drop = True)


### Cambiamos el tipo a object y verificamos

In [244]:
df_steam_games['id'] = df_steam_games['id'].astype(int)

In [245]:
df_steam_games_random = df_steam_games.sample(n=100, random_state =  102)  
id_y_precio_random = df_steam_games_random[['id','price']].head(10)
print(id_y_precio_random.dtypes)
id_y_precio_random


id        int32
price    object
dtype: object


,id,price
12353,768702,29.99
22482,427007,0.99
22891,455080,9.99
1415,218065,0.99
20464,537266,2.99
384,37390,6.99
12498,742380,4.99
11258,720100,4.99
10554,650760,16.99
2141,303332,12.99


Cambiamos el tipo 'id' a object y mostramos 10 ides al azar  

In [246]:
df_steam_games['id'] = df_steam_games['id'].astype(str) #

df_steam_games_random = df_steam_games.sample(n=100, random_state =  102)  
id_y_precio_random = df_steam_games_random[['id','price']].head(10)

print(id_y_precio_random.dtypes)
id_y_precio_random


id       object
price    object
dtype: object


,id,price
12353,768702,29.99
22482,427007,0.99
22891,455080,9.99
1415,218065,0.99
20464,537266,2.99
384,37390,6.99
12498,742380,4.99
11258,720100,4.99
10554,650760,16.99
2141,303332,12.99


## Tratamiento del campo 'price'


### Exploramos 'price'

In [247]:
print(df_steam_games.shape)
print('columnas en df_steam_games: ','[' + ', '.join(df_steam_games.keys()) + ']')
print("Tipo de dato en 'price':",df_steam_games['price'].dtype)

(30755, 13)
columnas en df_steam_games:  [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Tipo de dato en 'price': object


Mostramos los valores unicos de 'price'

In [248]:
unique_prices = df_steam_games['price'].unique()
print("Número de valores únicos en 'price': ", len(unique_prices))
print(unique_prices)

Número de valores únicos en 'price':  162
[4.99 'Free To Play' 'Free to Play' 0.99 2.99 3.99 9.99 18.99 29.99 'Free'
 10.99 1.5899999999999999 14.99 1.99 59.99 8.99 6.99 7.99 39.99 19.99 7.49
 12.99 5.99 2.49 15.99 1.25 24.99 17.99 61.99 3.49 11.99 13.99 'Free Demo'
 'Play for Free!' 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 'Install Now'
 21.99 89.99 'Play WARMACHINE: Tactics Demo' 0.98 139.92 4.29 64.99
 'Free Mod' 54.99 74.99 'Install Theme' 0.89 'Third-party' 0.5 'Play Now'
 299.99 1.29 3.0 15.0 5.49 23.99 49.0 20.99 10.93 1.3900000000000001
 'Free HITMAN™ Holiday Pack' 36.99 4.49 2.0 4.0 9.0 234.99
 1.9500000000000002 1.5 199.0 189.0 6.66 27.99 10.49 129.99 179.0 26.99
 399.99 31.99 399.0 20.0 40.0 3.33 199.99 22.99 320.0 38.85 71.7 59.95
 995.0 27.49 3.39 6.0 19.95 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3
 0.9500000000000001 172.24 249.99 2.9699999999999998 10.96 10.0 30.0 2.66
 6.48 19.29 11.15 18.9 2.89 'Pl

Vemos datos no numericos innecesarios y valores numericos derrapando

### Tratamiento de valores invalidos 

Redondeamos a dos decimales los numericos derrapados

In [249]:
# Función personalizada para redondear valores a dos decimales
def redondear_a_dos_decimales(valor):
    if isinstance(valor, (float, int)):
        return round(valor, 2)
    else:
        return valor  # Mantener valores no numéricos

# Aplica la función personalizada a la columna 'price'
df_steam_games['price'] = df_steam_games['price'].apply(redondear_a_dos_decimales)

unique_prices = df_steam_games['price'].unique()
print("Número de valores únicos en 'prices': ", len(unique_prices))
print(unique_prices)

Número de valores únicos en 'prices':  162
[4.99 'Free To Play' 'Free to Play' 0.99 2.99 3.99 9.99 18.99 29.99 'Free'
 10.99 1.59 14.99 1.99 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99 5.99
 2.49 15.99 1.25 24.99 17.99 61.99 3.49 11.99 13.99 'Free Demo'
 'Play for Free!' 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 'Install Now'
 21.99 89.99 'Play WARMACHINE: Tactics Demo' 0.98 139.92 4.29 64.99
 'Free Mod' 54.99 74.99 'Install Theme' 0.89 'Third-party' 0.5 'Play Now'
 299.99 1.29 3.0 15.0 5.49 23.99 49.0 20.99 10.93 1.39
 'Free HITMAN™ Holiday Pack' 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5 199.0
 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0 40.0
 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2.89 'Play the Demo' 99.0
 87.94 599.0 8.98 9.69 0.49 9.98 9.95 7.0 'Starting 

Convertimos el valor de todos los juegos gratuitos a 'Free To Play'

In [250]:
print(df_steam_games.shape)
filas_free_to_play = df_steam_games[df_steam_games['price'] == 'Free To Play'] 
print("Número de filas en 'Free to Play' antes de la conversion: ", filas_free_to_play.shape[0])

df_steam_games['price'].replace(['Free', 'Free Demo', 'Free to Play', 'Play for Free!', 'Free to Use', 'Free to Try', 'Play the Demo'], 'Free To Play', inplace=True)

filas_free_to_play = df_steam_games[df_steam_games['price'] == 'Free To Play'] 
print("Número de filas en 'Free to Play' despues de la conversion: ", filas_free_to_play.shape[0])

unique_prices = df_steam_games['price'].unique()
print("Número de valores únicos en 'prices': ", len(unique_prices))
print(unique_prices)

(30755, 13)


Número de filas en 'Free to Play' antes de la conversion:  462
Número de filas en 'Free to Play' despues de la conversion:  1895
Número de valores únicos en 'prices':  155
[4.99 'Free To Play' 0.99 2.99 3.99 9.99 18.99 29.99 10.99 1.59 14.99 1.99
 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99 5.99 2.49 15.99 1.25 24.99
 17.99 61.99 3.49 11.99 13.99 34.99 74.76 1.49 32.99 99.99 14.95 69.99
 16.99 79.99 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71
 'Install Now' 21.99 89.99 'Play WARMACHINE: Tactics Demo' 0.98 139.92
 4.29 64.99 'Free Mod' 54.99 74.99 'Install Theme' 0.89 'Third-party' 0.5
 'Play Now' 299.99 1.29 3.0 15.0 5.49 23.99 49.0 20.99 10.93 1.39
 'Free HITMAN™ Holiday Pack' 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5 199.0
 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0 40.0
 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2

Eliminamos todos los valores no numéricos en el campo 'price', con excepción de 'Free To Play'

In [251]:
#df = df_steam_games.copy()
# Checkeos previos
print("Número de filas en df antes de la eliminacion: ", df_steam_games.shape[0])
filas_free_to_play = df_steam_games[df_steam_games['price'] == 'Free To Play'] 
print("Número de filas en 'Free to Play' antes de la eliminacion: ", filas_free_to_play.shape[0])
print('\n')

# Mantener solo los valores numéricos y 'Free To Play'
df_steam_games['price'] = df_steam_games['price'].apply(lambda x: x if x == 'Free To Play' or re.match(r'^\d+(\.\d+)?$', str(x)) else None)

# Eliminar filas con valores nulos resultantes
df_steam_games.dropna(subset=['price'], inplace=True)

# Checkeos posteriores
print("Número de filas en df_steam_games despues de la eliminacion: ", df_steam_games.shape[0])
filas_free_to_play = df_steam_games[df_steam_games['price'] == 'Free To Play'] 
print("Número de filas en 'Free to Play' despues de la eliminacion: ", filas_free_to_play.shape[0])
print('\n')

unique_prices = df_steam_games['price'].unique()
print("Número de valores únicos en 'price': ", len(unique_prices))
print(unique_prices)

Número de filas en df antes de la eliminacion:  30755
Número de filas en 'Free to Play' antes de la eliminacion:  1895


Número de filas en df_steam_games despues de la eliminacion:  30740
Número de filas en 'Free to Play' despues de la eliminacion:  1895


Número de valores únicos en 'price':  145
[4.99 'Free To Play' 0.99 2.99 3.99 9.99 18.99 29.99 10.99 1.59 14.99 1.99
 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99 5.99 2.49 15.99 1.25 24.99
 17.99 61.99 3.49 11.99 13.99 34.99 74.76 1.49 32.99 99.99 14.95 69.99
 16.99 79.99 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 21.99
 89.99 0.98 139.92 4.29 64.99 54.99 74.99 0.89 0.5 299.99 1.29 3.0 15.0
 5.49 23.99 49.0 20.99 10.93 1.39 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5
 199.0 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0
 40.0 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2

Checkamos 10 registros aleatorios

In [252]:
muestra = df_steam_games.sample(n=100, random_state= 42)  
muestra[['id','price']].head(10)

,id,price
13892,731770,Free To Play
5104,442101,4.99
26083,222180,2.99
9994,463950,5.99
5429,439690,4.99
25986,314530,12.99
12894,733440,15.99
18203,513550,3.99
19387,523508,4.99
19158,569380,9.99


## Tratamiento del campo 'release_date'

### Exploramos 'release_date'

In [253]:
print(df_steam_games.shape)
print("Tipo de dato en 'release_date':",df_steam_games['release_date'].dtype)
print('columnas en df_steam_games: ','[' + ', '.join(df_steam_games.keys()) + ']')
print("Cantidad de valores faltantes en 'release_date':",df_steam_games['release_date'].isnull().sum())

#print(df_steam_games_prepared.isnull().sum())


(30740, 13)
Tipo de dato en 'release_date': object
columnas en df_steam_games:  [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]
Cantidad de valores faltantes en 'release_date': 1936


Vemos 1936 valores faltantes

Buscamos todos los valores unicos en 'release_date' y pintamos 50 de ellos

In [254]:
unique_values = df_steam_games['release_date'].unique() # captura los valores únicos en la columna 'release_date'
print("Número de valores únicos en 'release_date': ", len(unique_values))
print('\n')

print("Valores unicos aleatorios:")

# Selecciona 50 índices aleatorios
np.random.seed(90) 
indices_aleatorios = np.random.choice(len(unique_values), size=50, replace=False)

# Convierte la serie en un arreglo 10x5
matriz = np.array(unique_values[indices_aleatorios]).reshape(10, 5) 
print(matriz)

Número de valores únicos en 'release_date':  3446


Valores unicos aleatorios:
[['2012-12-17' '2013-09-20' '2013-07-30' '2012-07-25' '2006-06-08']
 ['2017-08-08' '2017-07-04' '2017-04-01' '2005-04-05' '2017-03-24']
 ['2017-07-24' '2012-02-23' '1994-05-05' '2013-07-31' '2011-05-05']
 ['2013-01-12' '2016-11-14' '2010-06-08' '2002-11-09' '2017-06-21']
 ['2015-09-05' '2015-04-15' '2013-05-20' '2014-01-13' '1998-01-31']
 ['2013-03-21' '2016-07-15' '1997-12-02' 'Aug 2015' '2012-10-29']
 ['2016-02-01' '2010-06-04' '2015-01-23' '2014-09-01' '2012-07-31']
 ['2005-07-21' '2011-04-20' '2002-11-18' '2011-11-09' 'Apr 2015']
 ['2007-10-27' '2016-03-02' '2017-07-12' '2015-06-23' '2014-05-09']
 ['2003-09-25' '2013-06-28' '2006-12-12' '2011-07-05' '2001-11-13']]


Vemos que existen valores sin el formato de fecha correcto como 'Aug 2015' 'Apr 2015'  
Pintamos los valores no validos en 'release_date'

In [255]:
# Define una expresión regular para el formato 'YYYY-MM-DD'
fecha_correcta_regex = r'\d{4}-\d{2}-\d{2}'

# Encuentra todos los valores únicos que no coinciden con el formato correcto
valores_no_validos = df_steam_games[~df_steam_games['release_date'].str.match(fecha_correcta_regex, na=False)]['release_date']
valores_no_validos_unicos = valores_no_validos.unique()
print("Cantidad de valores totales no validos en 'release_date': ", len(valores_no_validos))
print("Cantidad de valores no validos unicos en 'release_date': ", len(valores_no_validos_unicos))
print('\n')
# Imprime los valores únicos no válidos
print(valores_no_validos_unicos)


Cantidad de valores totales no validos en 'release_date':  1975
Cantidad de valores no validos unicos en 'release_date':  34


[None 'Jun 2009' 'Oct 2010' 'Feb 2011' 'Sep 2014' 'Apr 2015' 'Apr 2016'
 'Jul 2016' 'Jul 2017' 'SOON' '2018' 'Apr 2017' 'Jan 2017' 'Nov 2016'
 'Oct 2016' 'Jun 2016' 'Aug 2015' 'Jun 2015' 'May 2015' 'Feb 2015'
 'Jan 2015' 'Nov 2014' 'Aug 2014' 'Jul 2014' 'May 2014' 'Feb 2013'
 'Dec 2012' 'Jul 2010' 'Mar 2010' 'Jan 2010' 'Oct 2009' 'Sep 2009'
 'coming soon' 'SOON™']


Contamos 39 valores no validos en 'release_date'

### Eliminacion de valores faltantes

In [256]:
df_steam_games.dropna(subset=['release_date'], inplace=True)
print(df_steam_games.shape)
print("Cantidad de valores faltantes en 'release_date':",df_steam_games['release_date'].isnull().sum())

(28804, 13)
Cantidad de valores faltantes en 'release_date': 0


### Tratamiento de valores invalidos

Eliminamos los valores no validos tipo 'Feb 2013', 'SOON™', etc

In [257]:
print('Numero de filas y campos antes de la eliminacion: ',df_steam_games.shape)
# Define una expresión regular para el formato 'YYYY-MM-DD'
fecha_correcta_regex = r'\d{4}-\d{2}-\d{2}'

# Filtra el DataFrame para mantener solo las filas con fechas válidas
df_steam_games = df_steam_games[df_steam_games['release_date'].str.match(fecha_correcta_regex, na=False)]

print('Numero de filas y campos despues de la eliminacion: ',df_steam_games.shape)

Numero de filas y campos antes de la eliminacion:  (28804, 13)
Numero de filas y campos despues de la eliminacion:  (28765, 13)


## Guardamos el dataset preparado

In [258]:
df_steam_games.to_csv('../Data/df_steam_games_prepared.csv', index=False)

# CREACION DE LOS ENPOINT DATASETS

Terminada la preparacion de los datos en 'df_steam_games' toca crear los diferentes datasets en formato parquet que van alimentar al servidor de consultas

## Endpoint `'developer(desarrollador: str)'`

### Transformacion

In [ ]:
print('columnas en df_steam_games: ','[' + ', '.join(df_steam_games.keys()) + ']')

columnas en df_steam_games:  [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]


Filtramos los campos 'id' y 'developer'. Eliminamos el resto

In [ ]:
columnas_a_mantener = ['developer','release_date','genres']
df_steam_games_developer_desarrollador = df_steam_games[columnas_a_mantener]
df_steam_games_developer_desarrollador = df_steam_games_developer_desarrollador.copy()

df_steam_games_developer_desarrollador['genres'] = df_steam_games_developer_desarrollador['genres'].apply(lambda x: f"'{x}'")
df_steam_games_developer_desarrollador['genres'] = df_steam_games_developer_desarrollador['genres'].str.strip("'")

print(df_steam_games_developer_desarrollador.shape)
print(df_steam_games_developer_desarrollador.dtypes)
df_steam_games_developer_desarrollador.head(3)

(28765, 3)
developer       object
release_date    object
genres          object
dtype: object


,developer,release_date,genres
0,Kotoshiro,2018-01-04,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,Secret Level SRL,2018-01-04,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,Poolians.com,2017-07-24,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."


### To Parquet

In [ ]:
df_steam_games_developer_desarrollador.to_parquet('../Data/df_steam_games_developer_desarrollador.parquet', index=False)

## Endpoint `'def userdata(user_id:str)'`

### Transformacion

Filtramos los campos 'id' y 'price'. Eliminamos el resto

In [259]:
#df = df_steam_games.
columnas_a_mantener = ['id','price']
df_steam_games_userdata_user_id_respuesta1 = df_steam_games[columnas_a_mantener]
df_steam_games_userdata_user_id_respuesta1 = df_steam_games_userdata_user_id_respuesta1.copy()
df_steam_games_userdata_user_id_respuesta1['price'] = df_steam_games_userdata_user_id_respuesta1['price'].astype(str)

print(df_steam_games_userdata_user_id_respuesta1.shape)
print(df_steam_games_userdata_user_id_respuesta1.dtypes)
df_steam_games_userdata_user_id_respuesta1.head(3)

(28765, 2)
id       object
price    object
dtype: object


,id,price
0,761140,4.99
1,643980,Free To Play
2,670290,Free To Play


### To Parquet

In [260]:
df_steam_games_userdata_user_id_respuesta1.to_parquet('../Data/df_steam_games_userdata_user_id_respuesta1.parquet', index=False)

## Endpoint `'def UserForGenre(genero: str)'`

### Transformacion

Filtramos los campos 'id', 'genres' y 'release_date'. Eliminamos el resto

In [261]:
#df = df_steam_games.
columnas_a_mantener = ['id','genres','release_date']
df_steam_games_userforgenre_genero = df_steam_games[columnas_a_mantener]
df_steam_games_userforgenre_genero = df_steam_games_userforgenre_genero.copy()

df_steam_games_userforgenre_genero['genres'] = df_steam_games_userforgenre_genero['genres'].apply(lambda x: f"'{x}'")
df_steam_games_userforgenre_genero['genres'] = df_steam_games_userforgenre_genero['genres'].str.strip("'")

print(df_steam_games_userforgenre_genero.shape)
print(df_steam_games_userforgenre_genero.dtypes)
df_steam_games_userforgenre_genero.head(3)

(28765, 3)
id              object
genres          object
release_date    object
dtype: object


,id,genres,release_date
0,761140,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04
1,643980,"['Free to Play', 'Indie', 'RPG', 'Strategy']",2018-01-04
2,670290,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",2017-07-24


### To Parquet

In [262]:
df_steam_games_userforgenre_genero.to_parquet('../Data/df_steam_games_userforgenre_genero.parquet', index=False)

## Endpoint `'def best_developer_year(año : int)'`

### Transformacion

In [263]:
print('columnas en df_steam_games: ','[' + ', '.join(df_steam_games.keys()) + ']')

columnas en df_steam_games:  [publisher, genres, app_name, title, url, release_date, tags, reviews_url, specs, price, early_access, id, developer]


Filtramos los campos 'id' y 'developer'. Eliminamos el resto

In [264]:
columnas_a_mantener = ['id','developer']
df_steam_games_best_developer_year_año = df_steam_games[columnas_a_mantener]

print(df_steam_games_best_developer_year_año.shape)
print(df_steam_games_best_developer_year_año.dtypes)
df_steam_games_best_developer_year_año.head(3)

(28765, 2)
id           object
developer    object
dtype: object


,id,developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com


### To Parquet

In [265]:
df_steam_games_best_developer_year_año.to_parquet('../Data/df_steam_games_best_developer_year_año.parquet', index=False)

# Listar variables para eliminarlas

In [270]:
import builtins
import sys

# Obtener la lista de nombres de variables
variable_names = dir()
variables_descartables = ['exit','quit','Out','In','sys','var','var_name','variable_names','builtins','contenido','size','user_defined_variables','variables_descartables']
# Filtrar las variables creadas manualmente
user_defined_variables = [
    var for var in variable_names 
    #if not var.startswith('_')
    if var not in dir(builtins) and not var.startswith('_') and var not in variables_descartables
]
#print(user_defined_variables)
print('Variables encontradas: ', len(user_defined_variables))
# Imprimir el tamaño y la fecha de creación de las variables
for var_name in user_defined_variables:
    var = globals()[var_name]
    if sys.getsizeof(var) >= 100000:
        size = sys.getsizeof(var)  
        contenido = eval(var_name)
        print(f"La variable {var_name} pesa {size} bytes y es de tipo {str(type(contenido))[8:-2]}")

Variables encontradas:  31


La variable df pesa 6379904 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_steam_games pesa 29785679 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_steam_games_best_developer_year_año pesa 4107446 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_steam_games_random pesa 103380 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_steam_games_userdata_user_id_respuesta1 pesa 3813351 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_steam_games_userforgenre_genero pesa 6379904 bytes y es de tipo pandas.core.frame.DataFrame
La variable filas_free_to_play pesa 1941952 bytes y es de tipo pandas.core.frame.DataFrame
La variable ides pesa 492112 bytes y es de tipo pandas.core.series.Series
La variable ides_unicos pesa 246152 bytes y es de tipo numpy.ndarray
La variable muestra pesa 101047 bytes y es de tipo pandas.core.frame.DataFrame
